In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import os
from freezegun import freeze_time

from src.email_manager import mock_send_email
from src.lambda_function import run_weekly_update_process, run_mid_week_update_process
from src.supabase_client import mock_upsert_training_week_update, mock_upsert_training_week_with_coaching
from src.supabase_client import list_users
from src.auth_manager import get_strava_client

In [23]:
def setup_mock_user():
    users = list_users()
    mock_user_id = int(os.environ["JAMIES_ATHLETE_ID"])

    mock_user = None
    for user in users:
        if user.athlete_id == mock_user_id:
            mock_user = user
            break

    print(f"Mock user: {mock_user}")
    return mock_user

In [32]:
mock_user = setup_mock_user()

# re-authenticate the user if necessary prior to freeze_time
get_strava_client(mock_user.athlete_id)

@freeze_time("2024-08-18 20:00:00")
def mock_weekly_update():
    run_weekly_update_process(
        user=mock_user,
        upsert_fn=mock_upsert_training_week_with_coaching,
        email_fn=mock_send_email,
    )

@freeze_time("2024-08-21 20:00:00")
def mock_mid_week_update():
    run_mid_week_update_process(
        user=mock_user,
        upsert_fn=mock_upsert_training_week_update,
        email_fn=mock_send_email,
    )

Mock user: athlete_id=98390356 email='voynow99@gmail.com' preferences='A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays' created_at=datetime.datetime(2024, 8, 21, 21, 12, 55, 723179, tzinfo=datetime.timezone.utc)
athlete_id=98390356 token still valid until 2024-08-24 23:24:34+00:00


In [36]:
mock_weekly_update()

athlete_id=98390356 token still valid until 2024-08-24 23:24:34+00:00
{
    "athlete_id": 98390356,
    "training_week_planning": "To distribute the total weekly mileage of 44-46 miles, with a long run of 17-18 miles on Saturday, we can start by planning the rest of the week. If the long run is 18 miles, this leaves us with 26-28 miles for the other sessions. We want to have a speed workout on Wednesday and make Fridays a moderate run to prepare for the Saturday long run. Considering Monday and Thursday as easy runs for recovery, and Tuesday a rest day or a very light session if needed. Sunday is a rest day for full recovery.\n\n- Saturday: Long run = 18 miles\n- Wednesday: Speed workout (such as 5 miles of intervals) = 7 miles total\n- Monday: Easy run = 5 miles\n- Thursday: Easy run = 5 miles\n- Friday: Moderate run = 7 miles\n\nThis gives us: 18 (long) + 7 (speed) + 5 (easy) + 5 (easy) + 7 (moderate) = 42 miles, leaving us 2-4 miles to adjust over the week. Let's add another mile to

In [37]:
mock_mid_week_update()

athlete_id=98390356 token still valid until 2024-08-24 23:24:34+00:00
{
    "athlete_id": 98390356,
    "activities": [
        {
            "date": "Monday, August 19, 2024",
            "distance_in_miles": 13.15,
            "elevation_gain_in_feet": 1010.5,
            "pace_minutes_per_mile": 9.71
        },
        {
            "date": "Tuesday, August 20, 2024",
            "distance_in_miles": 0.0,
            "elevation_gain_in_feet": 0.0,
            "pace_minutes_per_mile": NaN
        },
        {
            "date": "Wednesday, August 21, 2024",
            "distance_in_miles": 2.02,
            "elevation_gain_in_feet": 0.0,
            "pace_minutes_per_mile": 10.12
        }
    ],
    "training_week": [
        {
            "day": "mon",
            "session_type": "easy run",
            "distance": 5.0,
            "notes": "Easy pace run to start the week off gently."
        },
        {
            "day": "tues",
            "session_type": "easy run",
        

In [2]:
from src.supabase_client import upsert_user
from src.types.user_row import UserRow


user = UserRow(
        athlete_id=None,
        email='voynow99@gmail.com',
        preferences="A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays",
    )
upsert_user(user)

athlete_id=98390356 email='voynow99@gmail.com' preferences='A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays' created_at=datetime.datetime(2024, 8, 25, 15, 44, 21, 474773)


APIResponse(data=[{'athlete_id': 98390356, 'created_at': '2024-08-25T15:44:21.474773+00:00', 'email': 'voynow99@gmail.com', 'preferences': 'A) Training for a marathon B) This will be my second marathon C) Prefer workouts on Wednesdays and long runs on Saturdays'}], count=None)